In [16]:
# =========================================================
# FULL CELL: PTB-XL (Kaggle) – Load ECG + cân bằng dữ liệu tối ưu
# =========================================================

!pip install -q wfdb

import wfdb
import pandas as pd
import numpy as np
import ast
import os
import gc
import h5py

from collections import Counter
from sklearn.model_selection import train_test_split

# =========================
# DATA PATH (KAGGLE)
# =========================
DATA_PATH = "/kaggle/input/ptb-xl-dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1"

# =========================
# LOAD METADATA
# =========================
df = pd.read_csv(f"{DATA_PATH}/ptbxl_database.csv")
scp = pd.read_csv(f"{DATA_PATH}/scp_statements.csv", index_col=0)

# =========================
# SUPERCLASS MAPPING (CHUẨN PAPER PTB-XL)
# =========================
def get_superclass(scp_codes_str):
    if pd.isna(scp_codes_str):
        return None
    try:
        scp_codes = ast.literal_eval(scp_codes_str)
    except:
        return None

    diagnostic_classes = []
    for code in scp_codes.keys():
        if code in scp.index and scp.loc[code, "diagnostic"] == 1:
            diagnostic_classes.append(scp.loc[code, "diagnostic_class"])

    # Ưu tiên chẩn đoán bệnh > normal
    for cls in ["MI", "STTC", "CD", "HYP", "NORM"]:
        if cls in diagnostic_classes:
            return cls
    return None

df["superclass"] = df["scp_codes"].apply(get_superclass)

CLASSES = ["NORM", "MI", "STTC", "CD", "HYP"]
df = df[df["superclass"].isin(CLASSES)].copy()

print("Phân bố lớp gốc:")
print(Counter(df["superclass"]))

# =========================
# LOAD ECG SIGNAL (FIX PATH)
# =========================
def load_ecg(filename_hr, filename_lr):
    path_hr = f"{DATA_PATH}/{filename_hr}"
    path_lr = f"{DATA_PATH}/{filename_lr}"

    if os.path.exists(path_hr + ".dat"):
        signal, _ = wfdb.rdsamp(path_hr)
    elif os.path.exists(path_lr + ".dat"):
        signal, _ = wfdb.rdsamp(path_lr)
        signal = np.repeat(signal, 5, axis=0)  # 100Hz → 500Hz
    else:
        return None

    # Chuẩn hoá độ dài = 10s (5000)
    if signal.shape[0] > 5000:
        signal = signal[:5000]
    elif signal.shape[0] < 5000:
        signal = np.pad(signal, ((0, 5000 - signal.shape[0]), (0, 0)))

    return signal.astype(np.float32)

# =========================
# LOAD ALL ECG
# =========================
print("\nLoading ECG signals (~15 phút)...")

X, y = [], []

for _, row in df.iterrows():
    sig = load_ecg(row["filename_hr"], row["filename_lr"])
    if sig is not None:
        X.append(sig)
        y.append(row["superclass"])

X = np.array(X)
y = np.array(y)

print("\nLoaded ECG:", X.shape)
print("Phân bố lớp sau load:", Counter(y))

# =========================
# CÂN BẰNG DỮ LIỆU (BEST PRACTICE)
# =========================
"""
Chiến lược:
- Giữ toàn bộ HYP (hiếm nhất)
- Các lớp lớn → cap ở mức hợp lý
- Không làm dataset quá nhỏ
"""

np.random.seed(42)

CAP = 2500   # bạn có thể tăng 3000–4000 nếu GPU đủ

X_bal, y_bal = [], []

for cls in CLASSES:
    idx = np.where(y == cls)[0]

    if len(idx) > CAP:
        idx = np.random.choice(idx, CAP, replace=False)

    X_bal.append(X[idx])
    y_bal.append(y[idx])

X_bal = np.concatenate(X_bal)
y_bal = np.concatenate(y_bal)

print("\nSau cân bằng (cap-based):")
print(Counter(y_bal))
print("Total samples:", len(y_bal))

# =========================
# SHUFFLE + SPLIT
# =========================
perm = np.random.permutation(len(y_bal))
X_bal = X_bal[perm]
y_bal = y_bal[perm]

X_train, X_val, y_train, y_val = train_test_split(
    X_bal,
    y_bal,
    test_size=0.2,
    stratify=y_bal,
    random_state=42
)

print("\nTrain:", X_train.shape)
print("Val  :", X_val.shape)

# =========================
# SAVE HDF5
# =========================
with h5py.File("ptbxl_train_balanced.h5", "w") as f:
    f.create_dataset("ecgs", data=X_train, compression="gzip")
    f.create_dataset("labels", data=y_train.astype("S"))

with h5py.File("ptbxl_val_balanced.h5", "w") as f:
    f.create_dataset("ecgs", data=X_val, compression="gzip")
    f.create_dataset("labels", data=y_val.astype("S"))

print("\n✅ FILE ĐÃ LƯU:")
print(" - ptbxl_train_balanced.h5")
print(" - ptbxl_val_balanced.h5")

# =========================
# CLEAN RAM
# =========================
del X, y, X_bal, y_bal, X_train, X_val, y_train, y_val
gc.collect()

print("\n🚀 DỮ LIỆU ĐÃ SẴN SÀNG TRAIN (CNN / Transformer ECG)")


Phân bố lớp gốc:
Counter({'NORM': 9083, 'MI': 5486, 'STTC': 3905, 'CD': 2418, 'HYP': 538})

Loading ECG signals (~15 phút)...

Loaded ECG: (21430, 5000, 12)
Phân bố lớp sau load: Counter({'NORM': 9083, 'MI': 5486, 'STTC': 3905, 'CD': 2418, 'HYP': 538})

Sau cân bằng (cap-based):
Counter({'NORM': 2500, 'MI': 2500, 'STTC': 2500, 'CD': 2418, 'HYP': 538})
Total samples: 10456

Train: (8364, 5000, 12)
Val  : (2092, 5000, 12)

✅ FILE ĐÃ LƯU:
 - ptbxl_train_balanced.h5
 - ptbxl_val_balanced.h5

🚀 DỮ LIỆU ĐÃ SẴN SÀNG TRAIN (CNN / Transformer ECG)


In [18]:
# =========================================================
# CELL 2: Advanced preprocessing + imbalance-aware training
# =========================================================

import h5py
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from scipy.signal import butter, filtfilt
from collections import Counter

# =========================
# LOAD DATA FROM HDF5
# =========================
with h5py.File("ptbxl_train_balanced.h5", "r") as f:
    X_train = f["ecgs"][:]              # (N, 5000, 12)
    y_train = f["labels"][:].astype(str)

with h5py.File("ptbxl_val_balanced.h5", "r") as f:
    X_val = f["ecgs"][:]
    y_val = f["labels"][:].astype(str)

print("Train shape:", X_train.shape)
print("Train class distribution:", Counter(y_train))

# =========================
# 1️⃣ BANDPASS FILTER (0.5–40 Hz)
# =========================
def bandpass_filter(ecg, low=0.5, high=40.0, fs=500, order=5):
    nyq = 0.5 * fs
    low /= nyq
    high /= nyq
    b, a = butter(order, [low, high], btype="band")
    return filtfilt(b, a, ecg, axis=0)

print("\nApplying bandpass filter...")
X_train = np.array([bandpass_filter(x) for x in X_train])
X_val   = np.array([bandpass_filter(x) for x in X_val])

# =========================
# 2️⃣ Z-SCORE NORMALIZATION (PER LEAD)
# =========================
def zscore_per_lead(ecg):
    mean = ecg.mean(axis=0, keepdims=True)
    std = ecg.std(axis=0, keepdims=True) + 1e-8
    return (ecg - mean) / std

print("Applying z-score normalization...")
X_train = np.array([zscore_per_lead(x) for x in X_train])
X_val   = np.array([zscore_per_lead(x) for x in X_val])

print("Preprocessing DONE")

# =========================
# LABEL ENCODING
# =========================
CLASSES = ["NORM", "MI", "STTC", "CD", "HYP"]
label_to_idx = {c: i for i, c in enumerate(CLASSES)}

y_train_idx = np.array([label_to_idx[y] for y in y_train])
y_val_idx   = np.array([label_to_idx[y] for y in y_val])

# =========================
# 3️⃣ CLASS-WEIGHTED LOSS
# =========================
counter = Counter(y_train)
total = sum(counter.values())

class_weights = torch.tensor(
    [total / (len(CLASSES) * counter[c]) for c in CLASSES],
    dtype=torch.float32
)

print("\nClass weights:", class_weights)

criterion = nn.CrossEntropyLoss(weight=class_weights.cuda())

# =========================
# PYTORCH DATALOADER
# =========================
train_ds = TensorDataset(
    torch.tensor(X_train).float(),
    torch.tensor(y_train_idx).long()
)

val_ds = TensorDataset(
    torch.tensor(X_val).float(),
    torch.tensor(y_val_idx).long()
)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False)

# =========================
# 4️⃣ MIXUP ECG (TRAIN ONLY)
# =========================
def mixup_ecg(x, y, alpha=0.2):
    lam = np.random.beta(alpha, alpha)
    idx = torch.randperm(x.size(0)).cuda()
    mixed_x = lam * x + (1 - lam) * x[idx]
    return mixed_x, y, y[idx], lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

print("\n🚀 DataLoader + preprocessing + class-weight + mixup READY")
print("➡️ Cell này kết thúc, bạn có thể TRAIN MODEL NGAY")


Train shape: (8364, 5000, 12)
Train class distribution: Counter({'MI': 2000, 'NORM': 2000, 'STTC': 2000, 'CD': 1934, 'HYP': 430})

Applying bandpass filter...
Applying z-score normalization...
Preprocessing DONE

Class weights: tensor([0.8364, 0.8364, 0.8364, 0.8649, 3.8902])

🚀 DataLoader + preprocessing + class-weight + mixup READY
➡️ Cell này kết thúc, bạn có thể TRAIN MODEL NGAY


In [19]:
# =========================================================
# CELL: Patient-wise Train / Val / Test split (PTB-XL)
# =========================================================

from sklearn.model_selection import GroupShuffleSplit
from collections import Counter
import numpy as np

# df đã có cột 'patient_id' và 'superclass'
patients = df["patient_id"].values
labels = df["superclass"].values

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_val_idx, test_idx = next(gss.split(df, labels, groups=patients))

df_trainval = df.iloc[train_val_idx]
df_test = df.iloc[test_idx]

print("Test distribution:")
print(Counter(df_test["superclass"]))

# ---- split train / val ----
gss2 = GroupShuffleSplit(n_splits=1, test_size=0.125, random_state=42)
train_idx, val_idx = next(
    gss2.split(
        df_trainval,
        df_trainval["superclass"],
        groups=df_trainval["patient_id"]
    )
)

df_train = df_trainval.iloc[train_idx]
df_val = df_trainval.iloc[val_idx]

print("\nTrain distribution:")
print(Counter(df_train["superclass"]))

print("\nVal distribution:")
print(Counter(df_val["superclass"]))


Test distribution:
Counter({'NORM': 1808, 'MI': 1117, 'STTC': 793, 'CD': 475, 'HYP': 103})

Train distribution:
Counter({'NORM': 6341, 'MI': 3855, 'STTC': 2713, 'CD': 1704, 'HYP': 386})

Val distribution:
Counter({'NORM': 934, 'MI': 514, 'STTC': 399, 'CD': 239, 'HYP': 49})


In [22]:
# =========================================================
# CELL: SAFE TEST EXPORT (aligned with TRAIN pipeline)
# =========================================================

import wfdb
import numpy as np
import os
import h5py
import gc
from collections import Counter
from scipy.signal import butter, filtfilt

# =========================
# CONFIG (GIỐNG TRAIN)
# =========================
DATA_PATH = "/kaggle/input/ptb-xl-dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1"
FS = 500
TARGET_LEN = 5000
CLASSES = ["NORM", "MI", "STTC", "CD", "HYP"]

# =========================
# ECG LOADER (GIỐNG TRAIN)
# =========================
def load_ecg(filename_hr, filename_lr):
    path_hr = f"{DATA_PATH}/{filename_hr}"
    path_lr = f"{DATA_PATH}/{filename_lr}"

    if os.path.exists(path_hr + ".dat"):
        sig, _ = wfdb.rdsamp(path_hr)
    elif os.path.exists(path_lr + ".dat"):
        sig, _ = wfdb.rdsamp(path_lr)
        sig = np.repeat(sig, 5, axis=0)  # 100Hz → 500Hz
    else:
        return None

    # Fix length = 5000
    if sig.shape[0] > TARGET_LEN:
        sig = sig[:TARGET_LEN]
    elif sig.shape[0] < TARGET_LEN:
        sig = np.pad(sig, ((0, TARGET_LEN - sig.shape[0]), (0, 0)))

    return sig.astype(np.float32)  # ÉP FLOAT32 NGAY TỪ ĐẦU

# =========================
# BANDPASS FILTER (0.5–40Hz)
# =========================
def bandpass_filter(ecg, low=0.5, high=40.0, fs=500, order=5):
    nyq = 0.5 * fs
    b, a = butter(order, [low / nyq, high / nyq], btype="band")
    ecg = filtfilt(b, a, ecg, axis=0)
    return ecg.astype(np.float32)  # ÉP LẠI FLOAT32

# =========================
# Z-SCORE PER LEAD
# =========================
def zscore_per_lead(ecg):
    mean = ecg.mean(axis=0, keepdims=True)
    std = ecg.std(axis=0, keepdims=True) + 1e-8
    ecg = (ecg - mean) / std
    return ecg.astype(np.float32)  # ÉP FLOAT32

# =========================
# LOAD + PREPROCESS TEST ECG
# =========================
print("Loading & preprocessing TEST ECG (SAFE MODE)...")

X_test, y_test = [], []

for _, row in df_test.iterrows():
    sig = load_ecg(row["filename_hr"], row["filename_lr"])
    if sig is None:
        continue

    sig = bandpass_filter(sig)
    sig = zscore_per_lead(sig)

    X_test.append(sig)
    y_test.append(row["superclass"])

X_test = np.stack(X_test).astype(np.float32)
y_test = np.array(y_test)

print("\nTEST SHAPE:", X_test.shape)
print("TEST CLASS DISTRIBUTION:", Counter(y_test))
print("TEST DTYPE:", X_test.dtype)

# =========================
# SAVE HDF5 (GIỐNG TRAIN)
# =========================
with h5py.File("ptbxl_test.h5", "w") as f:
    f.create_dataset("ecgs", data=X_test, compression="gzip")
    f.create_dataset("labels", data=y_test.astype("S"))

print("\n✅ SAVED: ptbxl_test.h5 (SAFE & ALIGNED)")

# =========================
# CLEAN RAM
# =========================
del X_test, y_test
gc.collect()

print("\n🚀 TEST SET READY — GUARANTEED NO LEAK / NO DTYPE BUG")


Loading & preprocessing TEST ECG (SAFE MODE)...

TEST SHAPE: (4296, 5000, 12)
TEST CLASS DISTRIBUTION: Counter({'NORM': 1808, 'MI': 1117, 'STTC': 793, 'CD': 475, 'HYP': 103})
TEST DTYPE: float32

✅ SAVED: ptbxl_test.h5 (SAFE & ALIGNED)

🚀 TEST SET READY — GUARANTEED NO LEAK / NO DTYPE BUG


In [23]:
import h5py

with h5py.File("ptbxl_test.h5", "r") as f:
    print(f["ecgs"].dtype)
    print(f["ecgs"].shape)


float32
(4296, 5000, 12)


In [21]:
# =========================================================
# CELL 1: Setup & DataLoader
# =========================================================
import h5py
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32
EPOCHS = 50
PATIENCE = 7

CLASSES = ["NORM", "MI", "STTC", "CD", "HYP"]
NUM_CLASSES = len(CLASSES)

label_encoder = LabelEncoder()
label_encoder.fit(CLASSES)

class ECGDataset(Dataset):
    def __init__(self, h5_path):
        with h5py.File(h5_path, "r") as f:
            self.X = f["ecgs"][:]
            self.y = f["labels"][:].astype(str)
        self.y = label_encoder.transform(self.y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx]).float().permute(1, 0)  # (C, T)
        y = torch.tensor(self.y[idx]).long()
        return x, y

train_loader = DataLoader(ECGDataset("ptbxl_train_balanced.h5"),
                          batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(ECGDataset("ptbxl_val_balanced.h5"),
                          batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(ECGDataset("ptbxl_test.h5"),
                          batch_size=BATCH_SIZE, shuffle=False)

print("DataLoader READY | Device:", DEVICE)


DataLoader READY | Device: cuda


In [30]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
import h5py

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)


Using device: cuda


In [32]:
from torch.utils.data import Dataset
import torch
import h5py

CLASSES = ["NORM", "MI", "STTC", "CD", "HYP"]
label_to_idx = {c: i for i, c in enumerate(CLASSES)}

class ECGDataset(Dataset):
    def __init__(self, h5_path):
        with h5py.File(h5_path, "r") as f:
            self.X = f["ecgs"][:]
            self.y = f["labels"][:]

        self.X = torch.tensor(self.X, dtype=torch.float32)

        # ---- FIX LABEL STRING -> INT ----
        self.y = torch.tensor(
            [label_to_idx[lbl.decode()] for lbl in self.y],
            dtype=torch.long
        )

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
train_ds = ECGDataset("ptbxl_train_balanced.h5")
val_ds   = ECGDataset("ptbxl_val_balanced.h5")

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False)

print("Train samples:", len(train_ds))
print("Val samples:", len(val_ds))


Train samples: 8364
Val samples: 2092


In [33]:
from collections import Counter

def mixup_ecg(x, y, alpha=0.2):
    if alpha <= 0:
        return x, y, y, 1.0

    lam = np.random.beta(alpha, alpha)
    idx = torch.randperm(x.size(0)).to(x.device)

    return lam*x + (1-lam)*x[idx], y, y[idx], lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam*criterion(pred, y_a) + (1-lam)*criterion(pred, y_b)


# ---- class weights ----
labels = [y.item() for _, y in train_ds]
counter = Counter(labels)
total = sum(counter.values())

weights = torch.tensor(
    [total/(len(counter)*counter[i]) for i in range(len(counter))],
    dtype=torch.float32
).to(DEVICE)

criterion = nn.CrossEntropyLoss(weight=weights)
print("Class weights:", weights)


Class weights: tensor([0.8364, 0.8364, 0.8364, 0.8649, 3.8902], device='cuda:0')


In [34]:
class Xception1D(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(12, 32, 3, padding=1),
            nn.ReLU(),
            nn.Conv1d(32, 64, 3, padding=1, groups=32),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        x = x.permute(0,2,1)
        x = self.conv(x).squeeze(-1)
        return self.fc(x)


In [35]:
models = {
    "Xception1D": Xception1D().to(DEVICE),
    "ResNet1D":   ResNet1D().to(DEVICE),
    "CRNN1D":     CRNN1D().to(DEVICE),
}


In [36]:
def train_model(model, name, epochs=50, patience=7):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    history = {
        "train_loss": [], "val_loss": [],
        "train_acc": [],  "val_acc": []
    }

    best_val = 1e9
    counter = 0

    for epoch in range(1, epochs+1):

        # ---- TRAIN ----
        model.train()
        tloss, yt, yp = [], [], []

        for x,y in train_loader:
            x,y = x.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()

            x, y_a, y_b, lam = mixup_ecg(x,y)
            out = model(x)
            loss = mixup_criterion(criterion, out, y_a, y_b, lam)

            loss.backward()
            optimizer.step()

            tloss.append(loss.item())
            yt.extend(y.cpu().numpy())
            yp.extend(out.argmax(1).cpu().numpy())

        train_loss = np.mean(tloss)
        train_acc = accuracy_score(yt, yp)

        # ---- VAL ----
        model.eval()
        vloss, yt, yp = [], [], []

        with torch.no_grad():
            for x,y in val_loader:
                x,y = x.to(DEVICE), y.to(DEVICE)
                out = model(x)
                loss = criterion(out,y)

                vloss.append(loss.item())
                yt.extend(y.cpu().numpy())
                yp.extend(out.argmax(1).cpu().numpy())

        val_loss = np.mean(vloss)
        val_acc = accuracy_score(yt, yp)

        print(
            f"{name} | Epoch {epoch:02d} | "
            f"Train loss {train_loss:.4f} | Train acc {train_acc:.4f} | "
            f"Val loss {val_loss:.4f} | Val acc {val_acc:.4f}"
        )

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)

        # ---- EARLY STOP ----
        if val_loss < best_val:
            best_val = val_loss
            counter = 0
            torch.save(model.state_dict(), f"{name}.pt")
        else:
            counter += 1
            if counter >= patience:
                print(f"⏹ Early stopping {name}")
                break

    return history


In [37]:
histories = {}

for name, model in models.items():
    print("\n" + "="*60)
    histories[name] = train_model(model, name)



Xception1D | Epoch 01 | Train loss 1.5977 | Train acc 0.2502 | Val loss 1.5655 | Val acc 0.1420
Xception1D | Epoch 02 | Train loss 1.5460 | Train acc 0.2704 | Val loss 1.5055 | Val acc 0.3533
Xception1D | Epoch 03 | Train loss 1.5071 | Train acc 0.3038 | Val loss 1.4661 | Val acc 0.3599
Xception1D | Epoch 04 | Train loss 1.4833 | Train acc 0.3016 | Val loss 1.4385 | Val acc 0.3939
Xception1D | Epoch 05 | Train loss 1.4516 | Train acc 0.3089 | Val loss 1.4216 | Val acc 0.4283
Xception1D | Epoch 06 | Train loss 1.4428 | Train acc 0.3092 | Val loss 1.4005 | Val acc 0.4235
Xception1D | Epoch 07 | Train loss 1.4313 | Train acc 0.3181 | Val loss 1.3917 | Val acc 0.4030
Xception1D | Epoch 08 | Train loss 1.4184 | Train acc 0.3075 | Val loss 1.3793 | Val acc 0.4316
Xception1D | Epoch 09 | Train loss 1.4192 | Train acc 0.3281 | Val loss 1.3676 | Val acc 0.4197
Xception1D | Epoch 10 | Train loss 1.4015 | Train acc 0.3305 | Val loss 1.3527 | Val acc 0.4340
Xception1D | Epoch 11 | Train loss 1.38

RuntimeError: Given groups=1, weight of size [64, 12, 7], expected input[32, 5000, 12] to have 12 channels, but got 5000 channels instead